In [8]:
from ipywidgets import widgets, HBox, VBox
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [13]:

# 设置中文字体
plt.rcParams['font.family'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 创建输入组件
l_text = widgets.FloatText(description='针长 l:', value=5.0, min=0.1)
d_text = widgets.FloatText(description='线间距 d:', value=10.0, min=0.1)
button = widgets.Button(description='生成 9 次投针结果')
output = widgets.Output()
stats_output = widgets.Output()  # 用于显示统计信息

# 定义按钮点击事件的回调函数
def on_button_clicked(b):
    with output, stats_output:
        clear_output()
        try:
            # 获取输入值
            l = float(l_text.value)
            d = float(d_text.value)
            
            # 验证输入
            if l <= 0 or d <= 0:
                print("错误：针长和线间距必须为正数！")
                return
            if l >= d:
                print("错误：针长必须小于线间距！")
                return
            
            # 计算理论概率
            theoretical_probability = (2 * l) / (np.pi * d)
            
            # 固定边界范围
            x_min_fixed = 0
            x_max_fixed = d * 3
            y_min_fixed = -l
            y_max_fixed = l + d
            
            # 创建 9 张子图 (3x3)
            fig, axes = plt.subplots(3, 3, figsize=(15, 12))  # 3 行 3 列布局
            axes = axes.flatten()  # 转换为一维数组方便遍历
            
            # 记录相交次数
            cross_count = 0
            
            for i, ax in enumerate(axes):
                # 随机生成一个针的位置和角度
                phi = np.random.uniform(0, np.pi)
                x_center = np.random.uniform(d, d*2)
                y_center = np.random.uniform(0, d)
                
                # 计算针的两个端点
                x1 = x_center - (l/2) * np.cos(phi)
                y1 = y_center - (l/2) * np.sin(phi)
                x2 = x_center + (l/2) * np.cos(phi)
                y2 = y_center + (l/2) * np.sin(phi)
                
                # 检查是否相交
                crosses = False
                for j in range(4):
                    if min(y1, y2) <= j*d <= max(y1, y2):
                        crosses = True
                        cross_count += 1
                        break
                
                # 绘制投针示意图
                ax.set_title(f'投针结果 {i+1}')
                ax.set_xlim(x_min_fixed, x_max_fixed)
                ax.set_ylim(y_min_fixed, y_max_fixed)
                ax.set_aspect('equal')
                ax.axis('on')  # 显示坐标轴
                
                # 绘制平行线
                for j in range(4):
                    ax.axhline(y=j*d, color='black', linestyle='-', linewidth=1)
                
                # 绘制针
                color = 'red' if crosses else 'blue'
                ax.plot([x1, x2], [y1, y2], color, linewidth=2)
                ax.plot(x_center, y_center, 'bo', markersize=5)
                
                # 绘制角度标记
                arc_radius = l/4
                arc_angles = np.linspace(0, phi, 100)
                arc_x = x_center + arc_radius * np.cos(arc_angles)
                arc_y = y_center + arc_radius * np.sin(arc_angles)
                ax.plot(arc_x, arc_y, 'g-', linewidth=1)
                ax.text(x_center + arc_radius*1.1, y_center + arc_radius*0.3, 
                        r'$\phi$', fontsize=12)
                
                # 添加相交状态说明
                status_y_pos = y_min_fixed + 0.1 * (y_max_fixed - y_min_fixed)
                ax.text(x_min_fixed + 0.1 * (x_max_fixed - x_min_fixed), status_y_pos, 
                        f"状态: {'相交' if crosses else '不相交'}", 
                        fontsize=10, bbox=dict(facecolor='white', alpha=0.8))
            
            plt.tight_layout()
            plt.show()
            
            # 显示理论概率和实际相交比例
            print(f"理论概率 P = {2*l/(np.pi*d):.4f}")
            print(f"实际相交比例 = {cross_count/9:.4f} ({cross_count}/9)")
            
        except ValueError:
            print("错误：请输入有效的数字！")

# 将按钮的点击事件与回调函数关联
button.on_click(on_button_clicked)

# 显示组件
display(VBox([HBox([l_text, d_text, button]), output, stats_output]))